In [1]:
import os, glob
import math, copy, time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from tqdm import tqdm, trange

import numpy as np

import matplotlib
from matplotlib import pyplot as plt

from IPython.display import HTML
from IPython.display import display

from skeleton_models import ntu_rgbd, ntu_ss_1, ntu_ss_2, ntu_ss_3
from graph import Graph
from render import animate, save_animation
from datasets import NTUDataset, Normalize, CropSequence, SelectDimensions, SelectSubSample

# Model components
from zoo_pose_embedding import TwoLayersGCNPoseEmbedding, JoaosDownsampling, SuperSimpleDownsampling
from zoo_action_encoder_units import TransformerEncoderUnit
from zoo_action_decoder_units import TransformerDecoderUnit
from zoo_upsampling import StepByStepUpsampling, JoaosUpsampling, SuperSimpleUpsampling
from model import ActionEmbeddingTransformer, SimplePoseEncoderDecoder
from layers import subsequent_mask

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
skeleton_model = ntu_rgbd
adjacency = Graph(skeleton_model)
conf_kernel_size = adjacency.A.shape[0]
conf_num_nodes = adjacency.A.shape[1]
conf_heads = 5
conf_encoding_per_node = 100
conf_internal_per_node = int(conf_encoding_per_node/conf_heads)
print(conf_encoding_per_node*conf_num_nodes)


class BetterThatBestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = ActionEmbeddingTransformer(
            JoaosDownsampling(
                conf_num_nodes,
                conf_encoding_per_node*conf_num_nodes,
                node_channel_in = 2,
                device=device
            ),
            TransformerEncoderUnit (
                heads=conf_heads,
                embedding_in=conf_num_nodes*conf_encoding_per_node,
                embedding_out=conf_num_nodes*conf_internal_per_node
            ),
            TransformerDecoderUnit(
                heads=conf_heads,
                embedding_in=conf_num_nodes*conf_encoding_per_node,
                embedding_out=conf_num_nodes*conf_internal_per_node,
                memory_in=conf_num_nodes*conf_encoding_per_node
            ),
            JoaosUpsampling(
                conf_num_nodes,
                conf_encoding_per_node*conf_num_nodes,
                node_channel_out = 2,
                device=device
            )
        )

    def forward(self, x_in, x_out, A, mask):
        return self.model(x_in, x_out, A, mask)
    
model = BetterThatBestModel()

A = torch.from_numpy(adjacency.A).to(device, dtype=torch.float)
model = model.to(device)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

criterion = torch.nn.L1Loss()

composed = transforms.Compose([Normalize(),
                               SelectDimensions(2),
                               SelectSubSample(skeleton_model)
                              ])

#ntu_dataset = NTUDataset(root_dir='../ntu-rgbd-dataset/Python/raw_npy/')
ntu_dataset = NTUDataset(root_dir='../datasets/NTURGB-D/Python/sel_npy/', transform=composed)
loader = DataLoader(ntu_dataset, batch_size=512, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)


pbar = tqdm(range(200000), desc='Initializing ...')
for epoch in pbar:
    
    for data in loader:
        data = data.to(device, dtype=torch.float)
        
        n_out, t_out, v_out, c_out = data.size()
        mask = subsequent_mask(t_out).to(device, dtype=torch.float)
        optimizer.zero_grad()
        out = model(data, data, A, mask)

        loss = criterion(out, data)
        loss.backward()

        # update parameters
        optimizer.step()
        pbar.set_description("Curr loss = {:.4f}".format(loss.item()))
        
    if epoch == 0:
        save_animation(data[0], skeleton_model, 'outputs/animations/a_sample_example_epoch_{}.gif'.format(epoch))
    
    if epoch % 250 == 0:
        print('Epoch {} loss = {}'.format(epoch, loss.item()))
        # torch.save(model.state_dict(), 'outputs/models/simple_encoder_epoch_{}.pth'.format(epoch))
        # save_animation(data[0], ntu_rgbd, 'outputs/animations/sample_example_epoch_{}.gif'.format(epoch))
        save_animation(out[0], skeleton_model, 'outputs/animations/out_example_epoch_{}.gif'.format(epoch))

2500
Warn: not activated
Warn: not activated


Curr loss = 0.7442:   0%|          | 0/200000 [00:00<?, ?it/s]

Epoch 0 loss = 0.7442317008972168


Curr loss = 0.0718:   0%|          | 250/200000 [08:02<105:00:57,  1.89s/it]

Epoch 250 loss = 0.07176075875759125


...

Epoch 7500 loss = 0.02736840397119522


Curr loss = 0.0263:   4%|▍         | 7750/200000 [3:46:25<83:57:48,  1.57s/it] 

Epoch 7750 loss = 0.026272529736161232


Curr loss = 0.0245:   4%|▍         | 7908/200000 [3:51:12<93:36:24,  1.75s/it] 


KeyboardInterrupt: 

In [ ]:
skeleton_model = ntu_rgbd
adjacency = Graph(skeleton_model)
conf_kernel_size = adjacency.A.shape[0]
conf_num_nodes = adjacency.A.shape[1]
conf_heads = 5
conf_encoding_per_node = 100
conf_internal_per_node = int(conf_encoding_per_node/conf_heads)
print(conf_encoding_per_node*conf_num_nodes)





In [ ]:
print('Using {}'.format(device))

model = SimplePoseEncoderDecoder(
    JoaosDownsampling(
        conf_num_nodes,
        conf_encoding_per_node*conf_num_nodes,
        node_channel_in = 2,
        device=device
    ),
    JoaosUpsampling(
        conf_num_nodes,
        conf_encoding_per_node*conf_num_nodes,
        node_channel_out = 2,
        device=device
    )
)

A = torch.from_numpy(adjacency.A).to(device, dtype=torch.float)
model = model.to(device)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

criterion = torch.nn.L1Loss()

composed = transforms.Compose([Normalize(),
                               SelectDimensions(2),
                               SelectSubSample(skeleton_model)
                              ])

#ntu_dataset = NTUDataset(root_dir='../ntu-rgbd-dataset/Python/raw_npy/')
ntu_dataset = NTUDataset(root_dir='../datasets/NTURGB-D/Python/sel_npy/', transform=composed)
loader = DataLoader(ntu_dataset, batch_size=512, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)


pbar = tqdm(range(200000), desc='Initializing ...')
for epoch in pbar:
    
    for data in loader:
        data = data.to(device, dtype=torch.float)

        n_out, c_out, t_out, v_out = data.size()
        mask = subsequent_mask(t_out).to(device, dtype=torch.float)
        optimizer.zero_grad()
        out = model(data, A)

        loss = criterion(out, data)
        loss.backward()

        # update parameters
        optimizer.step()
        pbar.set_description("Curr loss = {:.4f}".format(loss.item()))
        
    if epoch == 0:
        save_animation(data[0], skeleton_model, 'outputs/animations/a_sample_example_epoch_{}.gif'.format(epoch))
    
    if epoch % 3000 == 0:
        print('Epoch {} loss = {}'.format(epoch, loss.item()))
        # torch.save(model.state_dict(), 'outputs/models/simple_encoder_epoch_{}.pth'.format(epoch))
        # save_animation(data[0], ntu_rgbd, 'outputs/animations/sample_example_epoch_{}.gif'.format(epoch))
        save_animation(out[0], skeleton_model, 'outputs/animations/out_example_epoch_{}.gif'.format(epoch))

In [ ]:
print(ntu_dataset[0][:,:,1].min())
print(ntu_dataset[0][:,:,0].min())
print(ntu_dataset[0][:,:,1].max())
print(ntu_dataset[0][:,:,0].max())
print(ntu_dataset[0][:,:,1].mean())
print(ntu_dataset[0][:,:,0].mean())

In [ ]:
a = ntu_dataset[0]
a[:,:,0] = a[:,:,0] - a[:,:,0].min()
a[:,:,1] = a[:,:,1] - a[:,:,1].min()

In [ ]:
ntu_dataset[0].shape

In [ ]:
a[:,:,0].max()

In [ ]:
print(a[:,:,1].min())
print(a[:,:,0].min())
print(a[:,:,1].max())
print(a[:,:,0].max())
print(a[:,:,1].mean())
print(a[:,:,0].mean())